In [1]:
import os
import random
import numpy as np
import pandas as pd
import collections
import math
import nltk

In [2]:
#label class 

#theta
#theta_training_in_loop
#theta_testing_in_loop
#avg_weight

################################
#a function here on old code line 38 

#for testing part- line 22. turn that into an argmax to generate 
# predicted label 

In [14]:
class LabelClass:
    def __init__(self, label , number_of_zeros_needed):
        self.label= label 
        self.number_of_zeros_needed= number_of_zeros_needed

        self.theta = np.zeros(number_of_zeros_needed)
        self.running_total = np.zeros(number_of_zeros_needed)

    def calc_score(self, x_vector):
        return np.dot(x_vector, self.theta) 
    
    def running_total(self):
        self.running_total+=self.theta
    
    def calc_avg_weight(self, count):
        return self.running_total/count

    def is_label(self, label):
        return self.label = label

IndentationError: expected an indented block (3414117625.py, line 18)

In [6]:
uri_train  = 'https://raw.githubusercontent.com/thiagorainmaker77/liar_dataset/master/train.tsv'

df_train = pd.read_table(uri_train,
                             names = ['id',	'label'	,'statement',	'subject',	'speaker', 	'job', 	'state',	'party',	'barely_true_c',	'false_c',	'half_true_c',	'mostly_true_c',	'pants_on_fire_c',	'venue'])

labels= ['true', "false", "half-true", "barely-true"]
other_df = pd.DataFrame({"label": labels})

df_linear = df_train[df_train.label.isin(other_df.label)]

df_linear_1= df_linear.reset_index()
df_linear_1['statement'] = df_linear_1['statement'].str.lower()
#df_linear_1['statement']= df_linear_1['statement'].str.split()


In [7]:
uri_test  = 'https://raw.githubusercontent.com/thiagorainmaker77/liar_dataset/master/test.tsv'

df_test = pd.read_table(uri_train,
                             names = ['id',	'label'	,'statement',	'subject',	'speaker', 	'job', 	'state',	'party',	'barely_true_c',	'false_c',	'half_true_c',	'mostly_true_c',	'pants_on_fire_c',	'venue'])

other_df_1 = pd.DataFrame({"label": labels})

df_linear_test = df_test[df_test.label.isin(other_df_1.label)]

df_linear_test_1= df_linear_test.reset_index()
df_linear_test_1['statement'] = df_linear_test_1['statement'].str.lower()

In [8]:
df_linear_1['tokenized_statement'] = df_linear_1.apply(lambda row: nltk.word_tokenize(row['statement']), axis=1)
df_linear_test_1['tokenized_statement'] = df_linear_test_1.apply(lambda row: nltk.word_tokenize(row['statement']), axis=1)


In [9]:
#build a vocab 

#all the unique words in a dataset 
word_set = set()

for idx, row in df_linear_1.iterrows():
    word_set.update(row['tokenized_statement'])


In [10]:
#this is a look up table 
vocab = {word:idx for idx,word in enumerate(word_set)}

In [11]:
number_of_zeros_needed = len(vocab)

label_dict= {}

for i in labels: 

    label_dict[i]= LabelClass(i,number_of_zeros_needed)

In [12]:
def vector_generator(sequence_toekns, vocab): 

    return_vector= np.zeros(number_of_zeros_needed)

    #set some positions within that vector 
    for token in  sequence_toekns: 
        pos = vocab.get(token)
        if pos is not None:
            return_vector[pos] +=1



    return(return_vector)


In [36]:
def avg_perceptron(df_train, df_test):

    count = 0

    while count<100:

        df_train_sample= df_train.sample()

        #get us the lable 
        actual_label = df_train_sample["label"].iat[0]

        #one hot vector for sentence 
        x_vector= vector_generator(df_train_sample["tokenized_statement"].iat[0],vocab)

        #To James- why the .dot operation?? 

        #for i in range(len(theta)):

        #calc scores for algo 

        label_scores = []
        for label in labels:
            label_scores.append(label_dict[label].calc_score(x_vector))
        max_score_index = np.argmax(label_scores)
        predicted_label = labels[max_score_index]


        # predicted_label = "true" if theta_true_in_loop >= theta_false_in_loop else "false"

        if actual_label != predicted_label:

            print("wrong")
            for label_obj in label_dict:
                
            #To James- need more explanation on this part 
            if actual_label == "false":
                
                #34:08 
                #penalize if mistake and encourage if right 
                #we want theta false to go higher 

                theta_false = theta_false +x_vector #I want the actual label to have more weight on those words 
                theta_true = theta_true -x_vector 
            else: 
                theta_false = theta_false -x_vector 
                theta_true = theta_true +x_vector 
        else:
            print("right")

        avg_weight_true += theta_true
        avg_weight_false+= theta_false

        
        count+=1 


    return avg_weight_true/count, avg_weight_false/count

In [37]:
avg_weight_true,avg_weight_false= avg_perceptron(df_linear_1, df_linear_test_1)

right
right
wrong
wrong
right
wrong
right
right
right
wrong
wrong
right
wrong
right
right
right
right
right
wrong
right
wrong
wrong
wrong
right
wrong
right
wrong
wrong
wrong
wrong
right
right
wrong
right
right
right
wrong
right
wrong
right
right
wrong
wrong
wrong
wrong
right
right
right
wrong
right
wrong
wrong
right
right
right
wrong
right
right
wrong
wrong
right
wrong
wrong
wrong
right
right
right
wrong
right
wrong
wrong
right
right
right
right
wrong
wrong
wrong
wrong
right
right
wrong
wrong
wrong
right
right
wrong
wrong
right
right
wrong
right
wrong
right
right
wrong
right
wrong
right
right


In [38]:
df_test = df_linear_test_1

to_calc =[]


for i,row in df_test.iterrows():

    #get us the lable 
    actual_label = row["label"]

    #one hot vector for sentence 
    x_vector= vector_generator(row["tokenized_statement"],vocab)

    #To James- why the .dot operation?? 

    #for i in range(len(theta)):

    #calc scores for algo 
    theta_true_in_loop = np.dot(x_vector, avg_weight_true ) #averaged weight
    theta_false_in_loop= np.dot(x_vector, avg_weight_false)

    predicted_label = "true" if theta_true_in_loop >= theta_false_in_loop else "false"

    #this is to calc accurracy later 
    if predicted_label == row["label"]: 
        to_calc.append(1)
    else: 
        to_calc.append(0)

    ###########################################################################

count_correct = to_calc.count(1)
rows_count = df_linear_test_1.shape[0]
accuracy = count_correct/rows_count

print(accuracy)



0.5538000544810678
